Script to post-process high-resolution WRF model output.

Major tasks include computing the following for selected variables:
  1. domain-averages to produce time series
  2. vertical integrals
  3. pressure-level vertical interpolation

James Ruppert  
25 Nov 2024

In [1]:
from netCDF4 import Dataset
import numpy as np
from wrf import getvar, ALL_TIMES#, vinterp
import xarray as xr
from read_wrf_piccolo import *
from post_proc_functions import *
import os

In [2]:
do_2d_special = True
do_3d_vars = False
do_3d_special = False

In [3]:
########################################################
# Directories and test selection
########################################################

# datdir = "/glade/derecho/scratch/ruppert/piccolo/"
datdir = "/glade/campaign/univ/uokl0053/"
datdir = "/ourdisk/hpc/radclouds/auto_archive_notyet/tape_2copies/piccolo/"

case = "sept1-4"
test_process = "ctl"
wrf_dom = "wrf_fine"
nmem = 5 # number of ensemble members

########################################################
# New functions and single-use calls go here
########################################################

# Ens-member string tags (e.g., memb_01, memb_02, etc.)
memb0=1 # Starting member to read
memb_nums_str=np.arange(memb0,nmem+memb0,1).astype(str)
nustr = np.char.zfill(memb_nums_str, 2)
memb_all=np.char.add('memb_',nustr)

def memb_dir_settings(memb_dir):
    wrfdir = datdir+case+'/'+memb_dir+'/'+test_process+"/"+wrf_dom+"/"
    outdir = wrfdir+"post_proc/"
    os.makedirs(outdir, exist_ok=True)
    # Get WRF file list, dimensions
    wrffiles = get_wrf_file_list(wrfdir, "wrfout_d01*")
    # hffiles = get_wrf_file_list(wrfdir, "hfout_d01*")
    lat, lon, nx1, nx2, nz, npd = wrf_dims(wrffiles[0])
    nfiles = len(wrffiles)
    # New vertical dimension for pressure levels
    # dp = 25 # hPa
    # pres = np.arange(1000, 25, -dp)
    # nznew = len(pres)
    return outdir, wrffiles, nfiles, npd

In [4]:
def var_readcheck(varname, dict_in):
    # Using dictionary passing to get multiple uses of the same
    # variable wherever possible
    try:
    # Check if exists
        return dict_in[varname], dict_in
    except:
    # If not, create
        if varname == 'dp':
            try:
                dict_in[varname] = dict_in['pwrf'].differentiate('bottom_top')*-1
            except:
                dict_in['pwrf'] = getvar(dict_in['ds'], "p", units='Pa', timeidx=dict_in['timeidx']) # Pa
                dict_in[varname] = dict_in['pwrf'].differentiate('bottom_top')*-1 # Pa
        elif varname == 'qv':
            dict_in[varname] = getvar(dict_in['ds'], "QVAPOR", timeidx=dict_in['timeidx']) # kg/kg
        elif varname == 'pwrf':
            dict_in[varname] = getvar(dict_in['ds'], "p", units='Pa', timeidx=dict_in['timeidx']) # Pa
        elif varname == 'tmpk':
            dict_in[varname] = getvar(dict_in['ds'], "tk", timeidx=dict_in['timeidx']) # K
        return dict_in[varname], dict_in

In [5]:
# Read and reduce variables for a given time step
def get_2d_special_vars_it(ds, it_file, var_list):
    vars_it = {}
    dict_pass = {'ds': ds, 'timeidx': it_file}
    for ivar_str in var_list:
        if ivar_str == "pclass":
            dp, dict_pass = var_readcheck('dp', dict_pass)
            ivar = wrf_pclass(ds, dp, it_file)
        elif ivar_str == "pw":
            qv, dict_pass = var_readcheck('qv', dict_pass)
            ivar = vert_int(qv, dp)
        elif ivar_str == "pw_sat":
            tmpk, dict_pass = var_readcheck('tmpk', dict_pass)
            pwrf, dict_pass = var_readcheck('pwrf', dict_pass)
            ivar, dict_pass = rv_saturation(tmpk.values, pwrf.values) # kg/kg
        elif ivar_str == "vmf":
            wa = getvar(ds, "wa", timeidx=it_file)
            dp, dict_pass = var_readcheck('dp', dict_pass)
            ivar = vert_int(wa, dp)
        vars_it[ivar_str] = ivar
    return vars_it

# Loop over WRF input file time steps to read and reduce variables
def get_2d_special_vars_ifile(file, var_list):
    ds = Dataset(file)
    # Loop over dataset time steps
    nt_file = ds.dimensions['Time'].size
    vars_ifile = {}
    # for it_file in range(nt_file):
    for it_file in range(2):
        print("IT: ", it_file)
        vars_it = get_2d_special_vars_it(ds, it_file, var_list)
        for ivar_str in var_list:
            if it_file == 0:
                vars_ifile[ivar_str] = vars_it[ivar_str]
            else:
                vars_ifile[ivar_str] = xr.concat((vars_ifile[ivar_str], vars_it[ivar_str]), 'Time')
    ds.close()
    return vars_ifile

In [6]:
if do_2d_special:

    vars_2dspecial = ['pclass', 'pw', 'vmf']#, 'pw_sat']

    memb_dir = memb_all[0]

    # for memb_dir in memb_all:

    # memb_dir = memb_all[comm.rank]

    print("Processing "+memb_dir)
    outdir, wrffiles, nfiles, npd = memb_dir_settings(memb_dir)

    # Read in variable from WRF files
    vars_alltime = {}
    for ifile in range(nfiles):
    # for ifile in range(1):

        # Open the WRF file
        wrffile = wrffiles[ifile]
        print("Processing "+wrffile)

        # Get variables for entire file
        vars_ifile = get_2d_special_vars_ifile(wrffile, vars_2dspecial)

        # Concatenate variables
        for ivar_str in vars_2dspecial:
            if ifile == 0:
                vars_alltime[ivar_str] = vars_ifile[ivar_str]
            else:
                vars_alltime[ivar_str] = xr.concat((vars_alltime[ivar_str], vars_ifile[ivar_str]), 'Time')

    for ivar_str in vars_2dspecial:
        # Remove duplicate time steps
        vars_alltime[ivar_str] = vars_alltime[ivar_str].drop_duplicates(dim="Time", keep='first')
        # Write out the variables
        write_ncfile(outdir, vars_alltime[ivar_str], ivar_str)

    print("Done writing out special 2D variables")

Processing memb_01
Processing /ourdisk/hpc/radclouds/auto_archive_notyet/tape_2copies/piccolo/sept1-4/memb_01/ctl/wrf_fine/wrfout_d01_2024-09-01_12:00:00
IT:  0
GETTING DP
GETTING PWRF
GETTING QV
W:
Coordinates:
    XLONG    (south_north, west_east) float32 9MB -35.74 -35.73 ... -19.61
    XLAT     (south_north, west_east) float32 9MB 0.7503 0.7503 ... 16.64 16.64
    XTIME    float32 4B 0.0
    Time     datetime64[ns] 8B 2024-09-01T12:00:00
('bottom_top', 'south_north', 'west_east')
VMF:
Coordinates:
    XLONG    (south_north, west_east) float32 9MB -35.74 -35.73 ... -19.61
    XLAT     (south_north, west_east) float32 9MB 0.7503 0.7503 ... 16.64 16.64
    XTIME    float32 4B 0.0
    Time     datetime64[ns] 8B 2024-09-01T12:00:00
('south_north', 'west_east')
IT:  1
GETTING DP
GETTING PWRF
GETTING QV
W:
Coordinates:
    XLONG    (south_north, west_east) float32 9MB -35.74 -35.73 ... -19.61
    XLAT     (south_north, west_east) float32 9MB 0.7503 0.7503 ... 16.64 16.64
    XTIME    floa